In [25]:
# Importar las librerías necesarias
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score
)

# diccionario para guardar todas las métricas
metricas = {}

In [26]:
# --- 1. Carga del Conjunto de Datos ---
data = pd.read_csv("../data/subset/clean_subset_lifestyledata_rows5200_seed5200.csv")

# --- 2. Codificación de Etiquetas (Label Encoding) ---
label_encoder = LabelEncoder()
# Se transforma la variable objetivo 'Workout_Type' a valores numéricos.
data['Workout_Type'] = label_encoder.fit_transform(data['Workout_Type'])

# --- 3. Codificación One-Hot (One-Hot Encoding) ---
# Se define la lista de columnas categóricas nominales a transformar.
nominal_cols = ['Gender']
# sparse_output=False: Devuelve una matriz densa (array de NumPy) en lugar de una dispersa.
# handle_unknown='ignore': Si aparece una categoría no vista durante la transformación, la ignora.
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# Esto crea nuevas columnas binarias para cada categoría.
encoded = ohe.fit_transform(data[nominal_cols])
# Se convierte la matriz resultante en un DataFrame con nombres de columna apropiados.
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(nominal_cols))

# --- 4. Combinación de los Datos Procesados ---
# Se elimina la columna original 'Gender' del DataFrame principal.
# reset_index(drop=True) asegura que los índices se alineen correctamente para la concatenación.
data = data.drop(columns=nominal_cols).reset_index(drop=True)
encoded_df = encoded_df.reset_index(drop=True)

# Se concatenan el DataFrame original y el nuevo DataFrame con las columnas codificadas.
# axis=1 indica que la unión se realiza por columnas.
data = pd.concat([data, encoded_df], axis=1)

# --- 5. Visualización ---
data.head()

,Age,Weight_kg,Height_m,Max_BPM,Avg_BPM,Resting_BPM,Session_Duration_hours,Calories_Burned,Workout_Type,Gender_Female,Gender_Male
0,21.14,101.05,1.95,171.17,130.81,68.96,0.97,959.43,2,0.0,1.0
1,44.17,41.63,1.78,167.33,158.46,63.95,1.48,1424.35,0,0.0,1.0
2,20.07,63.81,1.78,187.86,137.11,60.93,1.70,1766.64,0,1.0,0.0
3,36.30,59.77,1.78,183.83,120.32,60.01,0.85,1028.50,1,1.0,0.0
4,51.99,57.60,1.56,166.25,151.82,67.97,1.66,1295.80,3,0.0,1.0


# Árbol de Decisiones:

#### 1. Árbol de Decisiones - CC:SI - ED:NO - Outliers:NO - Balanceo: NO

In [27]:
# datos a utilizar
data_model_1 = data.copy()

# Se definen las características (X) y la variable objetivo (y)
# X contiene todas las columnas EXCEPTO 'Workout_Type'.
X = data_model_1.drop("Workout_Type", axis=1)

# y contiene ÚNICAMENTE la columna 'Workout_Type'.
y = data_model_1["Workout_Type"]

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5200, stratify=y)

# Se crea un árbol con el criterio de impureza de Gini y una profundidad máxima de 5
modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=5200)
# Se entrena el modelo con los datos de entrenamiento
modelo_gini.fit(X_train, y_train)

# Se crea un árbol con el criterio de Entropía
modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=5200)
# Se entrena el modelo
modelo_entropy.fit(X_train, y_train)

# Se crea un árbol con Entropía y una condición de poda adicional
modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=5200)
# Se entrena el modelo
modelo_entropy_pruned.fit(X_train, y_train)

# Predecir con el modelo Gini
y_pred_gini = modelo_gini.predict(X_test)

# Predecir con el modelo de Entropía
y_pred_entropy = modelo_entropy.predict(X_test)

# Predecir con el modelo de Entropía podado
y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test)

# Métricas para el modelo Gini (accuracy, precision, recall, f1-score)
accuracy_gini = accuracy_score(y_test, y_pred_gini)
precision_gini = precision_score(y_test, y_pred_gini, average='weighted')
recall_gini = recall_score(y_test, y_pred_gini, average='weighted')
f1_gini = f1_score(y_test, y_pred_gini, average='weighted')

# Métricas para el modelo de Entropía (accuracy, precision, recall, f1-score)
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy, average='weighted')
recall_entropy = recall_score(y_test, y_pred_entropy, average='weighted')
f1_entropy = f1_score(y_test, y_pred_entropy, average='weighted')

# Métricas para el modelo de Entropía podado (accuracy, precision, recall, f1-score)
accuracy_entropy_pruned = accuracy_score(y_test, y_pred_entropy_pruned)
precision_entropy_pruned = precision_score(y_test, y_pred_entropy_pruned, average='weighted')
recall_entropy_pruned = recall_score(y_test, y_pred_entropy_pruned, average='weighted')
f1_entropy_pruned = f1_score(y_test, y_pred_entropy_pruned, average='weighted')

# Mostrar las métricas
print("Métricas del Modelo Gini:")
print(f"Accuracy: {accuracy_gini:.4f}, Precision: {precision_gini:.4f}, Recall: {recall_gini:.4f}, F1-Score: {f1_gini:.4f}\n")
print("Métricas del Modelo Entropía:")
print(f"Accuracy: {accuracy_entropy:.4f}, Precision: {precision_entropy:.4f}, Recall: {recall_entropy:.4f}, F1-Score: {f1_entropy:.4f}\n")
print("Métricas del Modelo Entropía Podado:")
print(f"Accuracy: {accuracy_entropy_pruned:.4f}, Precision: {precision_entropy_pruned:.4f}, Recall: {recall_entropy_pruned:.4f}, F1-Score: {f1_entropy_pruned:.4f}\n")



Métricas del Modelo Gini:
Accuracy: 0.7356, Precision: 0.7395, Recall: 0.7356, F1-Score: 0.7361

Métricas del Modelo Entropía:
Accuracy: 0.7750, Precision: 0.7953, Recall: 0.7750, F1-Score: 0.7820

Métricas del Modelo Entropía Podado:
Accuracy: 0.7750, Precision: 0.7953, Recall: 0.7750, F1-Score: 0.7820



In [28]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

## Conclusión:

Se evaluaron tres modelos de árbol de decisión: uno con Gini, otro con entropía y un tercero con entropía podado. Los dos modelos con entropía obtuvieron mejores resultados, con una precisión cercana al 78% y un F1-score de 0.78. Se seleccionó el modelo con entropía podado por ofrecer el mismo nivel de exactitud que el no podado, pero con menor complejidad y mejor capacidad de generalización, logrando un equilibrio óptimo entre rendimiento y simplicidad para predecir el tipo de entrenamiento.

#### 2. Árbol de Decisiones - CC:SI - ED:NO - Outliers:NO - Balanceo: SI

In [29]:
# datos a utilizar
data_model_2 = data.copy()

# Se definen las características (X) y la variable objetivo (y)
# X contiene todas las columnas EXCEPTO 'Workout_Type'.
X = data_model_2.drop("Workout_Type", axis=1)

# y contiene ÚNICAMENTE la columna 'Workout_Type'.
y = data_model_2["Workout_Type"]

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5200, stratify=y)

# Se crea un árbol con el criterio de impureza de Gini y una profundidad máxima de 5
modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=5200, class_weight='balanced')
# Se entrena el modelo con los datos de entrenamiento
modelo_gini.fit(X_train, y_train)

# Se crea un árbol con el criterio de Entropía
modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=5200, class_weight='balanced')
# Se entrena el modelo
modelo_entropy.fit(X_train, y_train)

# Se crea un árbol con Entropía y una condición de poda adicional
modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=5200, class_weight='balanced')
# Se entrena el modelo
modelo_entropy_pruned.fit(X_train, y_train)
    
# Predecir con el modelo Gini
y_pred_gini = modelo_gini.predict(X_test)

# Predecir con el modelo de Entropía
y_pred_entropy = modelo_entropy.predict(X_test)

# Predecir con el modelo de Entropía podado
y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test)

# Métricas para el modelo Gini (accuracy, precision, recall, f1-score)
accuracy_gini = accuracy_score(y_test, y_pred_gini)
precision_gini = precision_score(y_test, y_pred_gini, average='weighted')
recall_gini = recall_score(y_test, y_pred_gini, average='weighted')
f1_gini = f1_score(y_test, y_pred_gini, average='weighted')

# Métricas para el modelo de Entropía (accuracy, precision, recall, f1-score)
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy, average='weighted')
recall_entropy = recall_score(y_test, y_pred_entropy, average='weighted')
f1_entropy = f1_score(y_test, y_pred_entropy, average='weighted')

# Métricas para el modelo de Entropía podado (accuracy, precision, recall, f1-score)
accuracy_entropy_pruned = accuracy_score(y_test, y_pred_entropy_pruned)
precision_entropy_pruned = precision_score(y_test, y_pred_entropy_pruned, average='weighted')
recall_entropy_pruned = recall_score(y_test, y_pred_entropy_pruned, average='weighted')
f1_entropy_pruned = f1_score(y_test, y_pred_entropy_pruned, average='weighted')

# Mostrar las métricas
print("Métricas del Modelo Gini:")
print(f"Accuracy: {accuracy_gini:.4f}, Precision: {precision_gini:.4f}, Recall: {recall_gini:.4f}, F1-Score: {f1_gini:.4f}\n")
print("Métricas del Modelo Entropía:")
print(f"Accuracy: {accuracy_entropy:.4f}, Precision: {precision_entropy:.4f}, Recall: {recall_entropy:.4f}, F1-Score: {f1_entropy:.4f}\n")
print("Métricas del Modelo Entropía Podado:")
print(f"Accuracy: {accuracy_entropy_pruned:.4f}, Precision: {precision_entropy_pruned:.4f}, Recall: {recall_entropy_pruned:.4f}, F1-Score: {f1_entropy_pruned:.4f}\n")



Métricas del Modelo Gini:
Accuracy: 0.7337, Precision: 0.7392, Recall: 0.7337, F1-Score: 0.7348

Métricas del Modelo Entropía:
Accuracy: 0.7750, Precision: 0.7953, Recall: 0.7750, F1-Score: 0.7820

Métricas del Modelo Entropía Podado:
Accuracy: 0.7750, Precision: 0.7953, Recall: 0.7750, F1-Score: 0.7820



In [30]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

## Conclusión:

A pesar de aplicar el balanceo de clases mediante el parámetro class_weight='balanced', los resultados obtenidos fueron muy similares a los del modelo sin balancear. Esto indica que el conjunto de datos ya presentaba una distribución de clases relativamente equilibrada, por lo que el ajuste automático de pesos no generó cambios significativos en las métricas de desempeño. En consecuencia, el modelo mantiene su capacidad predictiva sin necesidad de un rebalanceo adicional, conservando una precisión y un F1-score estables alrededor del 0.78 en el modelo de entropía con podado.

#### 3. Árbol de Decisiones - CC:SI - ED:NO - Outliers:SI - Balanceo: NO

In [31]:
# datos a utilizar
data_model_3 = data.copy()

# ==========================
# Eliminar outliers (IQR)
# ==========================

# seleccionar solo las columnas numéricas
num_cols = data_model_3.select_dtypes(include=['float64', 'int64']).columns

# calcular Q1, Q3 y el rango intercuartílico (IQR)
Q1 = data_model_3[num_cols].quantile(0.25)
Q3 = data_model_3[num_cols].quantile(0.75)
IQR = Q3 - Q1

# crear una máscara booleana que identifique las filas SIN outliers
mask = ~((data_model_3[num_cols] < (Q1 - 1.5 * IQR)) |
         (data_model_3[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# filtrar los datos limpios
data_clean = data_model_3[mask].reset_index(drop=True)

print("Tamaño original:", data_model_3.shape)
print("Tamaño sin outliers:", data_clean.shape)

# ==========================
# Separar X e y
# ==========================
X = data_clean.drop("Workout_Type", axis=1)
y = data_clean["Workout_Type"]

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=5200, stratify=y
)

# Árbol con criterio Gini
modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=5200)
modelo_gini.fit(X_train, y_train)

# Árbol con criterio Entropía
modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=5200)
modelo_entropy.fit(X_train, y_train)

# Árbol con Entropía y poda
modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=5200)
modelo_entropy_pruned.fit(X_train, y_train)

# predicciones
y_pred_gini = modelo_gini.predict(X_test)
y_pred_entropy = modelo_entropy.predict(X_test)
y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test)

# métricas
accuracy_gini = accuracy_score(y_test, y_pred_gini)
precision_gini = precision_score(y_test, y_pred_gini, average='weighted')
recall_gini = recall_score(y_test, y_pred_gini, average='weighted')
f1_gini = f1_score(y_test, y_pred_gini, average='weighted')

accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy, average='weighted')
recall_entropy = recall_score(y_test, y_pred_entropy, average='weighted')
f1_entropy = f1_score(y_test, y_pred_entropy, average='weighted')

accuracy_entropy_pruned = accuracy_score(y_test, y_pred_entropy_pruned)
precision_entropy_pruned = precision_score(y_test, y_pred_entropy_pruned, average='weighted')
recall_entropy_pruned = recall_score(y_test, y_pred_entropy_pruned, average='weighted')
f1_entropy_pruned = f1_score(y_test, y_pred_entropy_pruned, average='weighted')

# ==========================
# Mostrar resultados
# ==========================
print("\nMétricas del Modelo Gini:")
print(f"Accuracy: {accuracy_gini:.4f}, Precision: {precision_gini:.4f}, Recall: {recall_gini:.4f}, F1-Score: {f1_gini:.4f}\n")

print("Métricas del Modelo Entropía:")
print(f"Accuracy: {accuracy_entropy:.4f}, Precision: {precision_entropy:.4f}, Recall: {recall_entropy:.4f}, F1-Score: {f1_entropy:.4f}\n")

print("Métricas del Modelo Entropía Podado:")
print(f"Accuracy: {accuracy_entropy_pruned:.4f}, Precision: {precision_entropy_pruned:.4f}, Recall: {recall_entropy_pruned:.4f}, F1-Score: {f1_entropy_pruned:.4f}\n")


Tamaño original: (5200, 11)
Tamaño sin outliers: (5055, 11)

Métricas del Modelo Gini:
Accuracy: 0.7883, Precision: 0.8018, Recall: 0.7883, F1-Score: 0.7908

Métricas del Modelo Entropía:
Accuracy: 0.7943, Precision: 0.8060, Recall: 0.7943, F1-Score: 0.7945

Métricas del Modelo Entropía Podado:
Accuracy: 0.7943, Precision: 0.8060, Recall: 0.7943, F1-Score: 0.7945



In [32]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

## Conclusión:

La eliminación de outliers mejoró las métricas del modelo, indicando que los datos atípicos afectaban su desempeño. Al limpiar el conjunto de datos, los árboles de decisión lograron una clasificación más precisa y estable.

#### 4. Árbol de Decisiones - CC:SI - ED:NO - Outliers:SI - Balanceo: SI

In [33]:
# datos a utilizar
data_model_4 = data.copy()

# ==========================
# Eliminar outliers (IQR)
# ==========================

# seleccionar solo las columnas numéricas
num_cols = data_model_4.select_dtypes(include=['float64', 'int64']).columns

# calcular Q1, Q3 y el rango intercuartílico (IQR)
Q1 = data_model_4[num_cols].quantile(0.25)
Q3 = data_model_4[num_cols].quantile(0.75)
IQR = Q3 - Q1

# crear una máscara booleana que identifique las filas SIN outliers
mask = ~((data_model_4[num_cols] < (Q1 - 1.5 * IQR)) |
         (data_model_4[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# filtrar los datos limpios
data_clean = data_model_4[mask].reset_index(drop=True)

print("Tamaño original:", data_model_4.shape)
print("Tamaño sin outliers:", data_clean.shape)

# ==========================
# Separar X e y
# ==========================
X = data_clean.drop("Workout_Type", axis=1)
y = data_clean["Workout_Type"]

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=5200, stratify=y
)

# Árbol con criterio Gini
modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=5200, class_weight='balanced')
modelo_gini.fit(X_train, y_train)

# Árbol con criterio Entropía
modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=5200, class_weight='balanced')
modelo_entropy.fit(X_train, y_train)

# Árbol con Entropía y poda
modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=5200, class_weight='balanced')
modelo_entropy_pruned.fit(X_train, y_train)

# predicciones
y_pred_gini = modelo_gini.predict(X_test)
y_pred_entropy = modelo_entropy.predict(X_test)
y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test)

# métricas
accuracy_gini = accuracy_score(y_test, y_pred_gini)
precision_gini = precision_score(y_test, y_pred_gini, average='weighted')
recall_gini = recall_score(y_test, y_pred_gini, average='weighted')
f1_gini = f1_score(y_test, y_pred_gini, average='weighted')

accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy, average='weighted')
recall_entropy = recall_score(y_test, y_pred_entropy, average='weighted')
f1_entropy = f1_score(y_test, y_pred_entropy, average='weighted')

accuracy_entropy_pruned = accuracy_score(y_test, y_pred_entropy_pruned)
precision_entropy_pruned = precision_score(y_test, y_pred_entropy_pruned, average='weighted')
recall_entropy_pruned = recall_score(y_test, y_pred_entropy_pruned, average='weighted')
f1_entropy_pruned = f1_score(y_test, y_pred_entropy_pruned, average='weighted')

# ==========================
# Mostrar resultados
# ==========================
print("\nMétricas del Modelo Gini:")
print(f"Accuracy: {accuracy_gini:.4f}, Precision: {precision_gini:.4f}, Recall: {recall_gini:.4f}, F1-Score: {f1_gini:.4f}\n")

print("Métricas del Modelo Entropía:")
print(f"Accuracy: {accuracy_entropy:.4f}, Precision: {precision_entropy:.4f}, Recall: {recall_entropy:.4f}, F1-Score: {f1_entropy:.4f}\n")

print("Métricas del Modelo Entropía Podado:")
print(f"Accuracy: {accuracy_entropy_pruned:.4f}, Precision: {precision_entropy_pruned:.4f}, Recall: {recall_entropy_pruned:.4f}, F1-Score: {f1_entropy_pruned:.4f}\n")


Tamaño original: (5200, 11)
Tamaño sin outliers: (5055, 11)

Métricas del Modelo Gini:
Accuracy: 0.7883, Precision: 0.8018, Recall: 0.7883, F1-Score: 0.7908

Métricas del Modelo Entropía:
Accuracy: 0.8002, Precision: 0.8114, Recall: 0.8002, F1-Score: 0.8018

Métricas del Modelo Entropía Podado:
Accuracy: 0.8002, Precision: 0.8114, Recall: 0.8002, F1-Score: 0.8018



In [34]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

## Conclusión:

Ahora, con el balanceo de clases y la eliminación de outliers, el modelo mostró una mejora notable en sus métricas.
Por ejemplo, el modelo con entropía alcanzó un accuracy de 0.8002, precisión de 0.8114, recall de 0.8002 y un F1-Score de 0.8018, superando los valores anteriores.
Esto demuestra que al equilibrar las clases y eliminar datos atípicos, el árbol de decisión logra una clasificación más precisa y estable.

#### 5. Árbol de Decisiones - CC:SI - ED:SI - Outliers:NO - Balanceo: NO

In [35]:
# datos a utilizar
data_model_5 = data.copy()

# Se definen las características (X) y la variable objetivo (y)
# X contiene todas las columnas EXCEPTO 'Workout_Type'.
X = data_model_5.drop("Workout_Type", axis=1)

# y contiene ÚNICAMENTE la columna 'Workout_Type'.
y = data_model_5["Workout_Type"]

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5200, stratify=y)

# 🔹 Escalado de las características (excepto 'Gender' y 'Workout_Type')
scaler = StandardScaler()

# Seleccionar columnas numéricas que no sean categóricas
numeric_cols = [col for col in X_train.columns if col not in ['Gender', 'Workout_Type']]

# Hacer copias para no perder las columnas categóricas
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Escalar solo las columnas numéricas
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Se crea un árbol con el criterio de impureza de Gini y una profundidad máxima de 5
modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=5200)
# Se entrena el modelo con los datos de entrenamiento
modelo_gini.fit(X_train_scaled, y_train)

# Se crea un árbol con el criterio de Entropía
modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=5200)
# Se entrena el modelo
modelo_entropy.fit(X_train_scaled, y_train)

# Se crea un árbol con Entropía y una condición de poda adicional
modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=5200)
# Se entrena el modelo
modelo_entropy_pruned.fit(X_train_scaled, y_train)

# Predecir con el modelo Gini
y_pred_gini = modelo_gini.predict(X_test_scaled)

# Predecir con el modelo de Entropía
y_pred_entropy = modelo_entropy.predict(X_test_scaled)

# Predecir con el modelo de Entropía podado
y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test_scaled)

# Métricas para el modelo Gini (accuracy, precision, recall, f1-score)
accuracy_gini = accuracy_score(y_test, y_pred_gini)
precision_gini = precision_score(y_test, y_pred_gini, average='weighted')
recall_gini = recall_score(y_test, y_pred_gini, average='weighted')
f1_gini = f1_score(y_test, y_pred_gini, average='weighted')

# Métricas para el modelo de Entropía (accuracy, precision, recall, f1-score)
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy, average='weighted')
recall_entropy = recall_score(y_test, y_pred_entropy, average='weighted')
f1_entropy = f1_score(y_test, y_pred_entropy, average='weighted')

# Métricas para el modelo de Entropía podado (accuracy, precision, recall, f1-score)
accuracy_entropy_pruned = accuracy_score(y_test, y_pred_entropy_pruned)
precision_entropy_pruned = precision_score(y_test, y_pred_entropy_pruned, average='weighted')
recall_entropy_pruned = recall_score(y_test, y_pred_entropy_pruned, average='weighted')
f1_entropy_pruned = f1_score(y_test, y_pred_entropy_pruned, average='weighted')

# Mostrar las métricas
print("Métricas del Modelo Gini:")
print(f"Accuracy: {accuracy_gini:.4f}, Precision: {precision_gini:.4f}, Recall: {recall_gini:.4f}, F1-Score: {f1_gini:.4f}\n")
print("Métricas del Modelo Entropía:")
print(f"Accuracy: {accuracy_entropy:.4f}, Precision: {precision_entropy:.4f}, Recall: {recall_entropy:.4f}, F1-Score: {f1_entropy:.4f}\n")
print("Métricas del Modelo Entropía Podado:")
print(f"Accuracy: {accuracy_entropy_pruned:.4f}, Precision: {precision_entropy_pruned:.4f}, Recall: {recall_entropy_pruned:.4f}, F1-Score: {f1_entropy_pruned:.4f}\n")



Métricas del Modelo Gini:
Accuracy: 0.7356, Precision: 0.7395, Recall: 0.7356, F1-Score: 0.7361

Métricas del Modelo Entropía:
Accuracy: 0.7750, Precision: 0.7953, Recall: 0.7750, F1-Score: 0.7820

Métricas del Modelo Entropía Podado:
Accuracy: 0.7750, Precision: 0.7953, Recall: 0.7750, F1-Score: 0.7820



In [ ]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

## Conclusión:

En este caso, la normalización de los datos no afectó el rendimiento del modelo, ya que los árboles de decisión no dependen de las escalas de las variables.
Esto se debe a que los árboles no calculan distancias ni magnitudes, sino que realizan divisiones basadas en umbrales de los valores de cada atributo para separar las clases.

#### 6. Árbol de Decisiones - CC:SI - ED:SI - Outliers:NO - Balanceo: SI

In [40]:
# datos a utilizar
data_model_6 = data.copy()

# Se definen las características (X) y la variable objetivo (y)
# X contiene todas las columnas EXCEPTO 'Workout_Type'.
X = data_model_6.drop("Workout_Type", axis=1)

# y contiene ÚNICAMENTE la columna 'Workout_Type'.
y = data_model_6["Workout_Type"]

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5200, stratify=y)

# 🔹 Escalado de las características (excepto 'Gender' y 'Workout_Type')
scaler = StandardScaler()

# Seleccionar columnas numéricas que no sean categóricas
numeric_cols = [col for col in X_train.columns if col not in ['Gender', 'Workout_Type']]

# Hacer copias para no perder las columnas categóricas
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Escalar solo las columnas numéricas
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Se crea un árbol con el criterio de impureza de Gini y una profundidad máxima de 5
modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=5200, class_weight='balanced')
# Se entrena el modelo con los datos de entrenamiento
modelo_gini.fit(X_train_scaled, y_train)

# Se crea un árbol con el criterio de Entropía
modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=5200, class_weight='balanced')
# Se entrena el modelo
modelo_entropy.fit(X_train_scaled, y_train)

# Se crea un árbol con Entropía y una condición de poda adicional
modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=5200, class_weight='balanced')
# Se entrena el modelo
modelo_entropy_pruned.fit(X_train_scaled, y_train)

# Predecir con el modelo Gini
y_pred_gini = modelo_gini.predict(X_test_scaled)

# Predecir con el modelo de Entropía
y_pred_entropy = modelo_entropy.predict(X_test_scaled)

# Predecir con el modelo de Entropía podado
y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test_scaled)

# Métricas para el modelo Gini (accuracy, precision, recall, f1-score)
accuracy_gini = accuracy_score(y_test, y_pred_gini)
precision_gini = precision_score(y_test, y_pred_gini, average='weighted')
recall_gini = recall_score(y_test, y_pred_gini, average='weighted')
f1_gini = f1_score(y_test, y_pred_gini, average='weighted')

# Métricas para el modelo de Entropía (accuracy, precision, recall, f1-score)
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy, average='weighted')
recall_entropy = recall_score(y_test, y_pred_entropy, average='weighted')
f1_entropy = f1_score(y_test, y_pred_entropy, average='weighted')

# Métricas para el modelo de Entropía podado (accuracy, precision, recall, f1-score)
accuracy_entropy_pruned = accuracy_score(y_test, y_pred_entropy_pruned)
precision_entropy_pruned = precision_score(y_test, y_pred_entropy_pruned, average='weighted')
recall_entropy_pruned = recall_score(y_test, y_pred_entropy_pruned, average='weighted')
f1_entropy_pruned = f1_score(y_test, y_pred_entropy_pruned, average='weighted')

# Mostrar las métricas
print("Métricas del Modelo Gini:")
print(f"Accuracy: {accuracy_gini:.4f}, Precision: {precision_gini:.4f}, Recall: {recall_gini:.4f}, F1-Score: {f1_gini:.4f}\n")
print("Métricas del Modelo Entropía:")
print(f"Accuracy: {accuracy_entropy:.4f}, Precision: {precision_entropy:.4f}, Recall: {recall_entropy:.4f}, F1-Score: {f1_entropy:.4f}\n")
print("Métricas del Modelo Entropía Podado:")
print(f"Accuracy: {accuracy_entropy_pruned:.4f}, Precision: {precision_entropy_pruned:.4f}, Recall: {recall_entropy_pruned:.4f}, F1-Score: {f1_entropy_pruned:.4f}\n")

Métricas del Modelo Gini:
Accuracy: 0.7337, Precision: 0.7392, Recall: 0.7337, F1-Score: 0.7348

Métricas del Modelo Entropía:
Accuracy: 0.7750, Precision: 0.7953, Recall: 0.7750, F1-Score: 0.7820

Métricas del Modelo Entropía Podado:
Accuracy: 0.7750, Precision: 0.7953, Recall: 0.7750, F1-Score: 0.7820



In [ ]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

## Conclusión:

El balanceo no generó cambios en las métricas, tanto con como sin normalización, ya que el conjunto de datos original ya estaba equilibrado entre clases. Por eso, el rendimiento del modelo se mantuvo prácticamente igual.

#### 7. Árbol de Decisiones - CC:SI - ED:SI - Outliers:SI - Balanceo: NO

In [37]:
# datos a utilizar
data_model_7 = data.copy()

# ==========================
# Eliminar outliers (IQR)
# ==========================

# seleccionar solo las columnas numéricas
num_cols = data_model_7.select_dtypes(include=['float64', 'int64']).columns

# calcular Q1, Q3 y el rango intercuartílico (IQR)
Q1 = data_model_7[num_cols].quantile(0.25)
Q3 = data_model_7[num_cols].quantile(0.75)
IQR = Q3 - Q1

# crear una máscara booleana que identifique las filas SIN outliers
mask = ~((data_model_7[num_cols] < (Q1 - 1.5 * IQR)) |
         (data_model_7[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# filtrar los datos limpios
data_clean = data_model_7[mask].reset_index(drop=True)

print("Tamaño original:", data_model_7.shape)
print("Tamaño sin outliers:", data_clean.shape)

# ==========================
# Separar X e y
# ==========================
X = data_clean.drop("Workout_Type", axis=1)
y = data_clean["Workout_Type"]

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5200, stratify=y)

# 🔹 Escalado de las características (excepto 'Gender' y 'Workout_Type')
scaler = StandardScaler()

# Seleccionar columnas numéricas que no sean categóricas
numeric_cols = [col for col in X_train.columns if col not in ['Gender', 'Workout_Type']]

# Hacer copias para no perder las columnas categóricas
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Escalar solo las columnas numéricas
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Se crea un árbol con el criterio de impureza de Gini y una profundidad máxima de 5
modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=5200)
# Se entrena el modelo con los datos de entrenamiento
modelo_gini.fit(X_train_scaled, y_train)

# Se crea un árbol con el criterio de Entropía
modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=5200)
# Se entrena el modelo
modelo_entropy.fit(X_train_scaled, y_train)

# Se crea un árbol con Entropía y una condición de poda adicional
modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=5200)
# Se entrena el modelo
modelo_entropy_pruned.fit(X_train_scaled, y_train)

# Predecir con el modelo Gini
y_pred_gini = modelo_gini.predict(X_test_scaled)

# Predecir con el modelo de Entropía
y_pred_entropy = modelo_entropy.predict(X_test_scaled)

# Predecir con el modelo de Entropía podado
y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test_scaled)

# Métricas para el modelo Gini (accuracy, precision, recall, f1-score)
accuracy_gini = accuracy_score(y_test, y_pred_gini)
precision_gini = precision_score(y_test, y_pred_gini, average='weighted')
recall_gini = recall_score(y_test, y_pred_gini, average='weighted')
f1_gini = f1_score(y_test, y_pred_gini, average='weighted')

# Métricas para el modelo de Entropía (accuracy, precision, recall, f1-score)
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy, average='weighted')
recall_entropy = recall_score(y_test, y_pred_entropy, average='weighted')
f1_entropy = f1_score(y_test, y_pred_entropy, average='weighted')

# Métricas para el modelo de Entropía podado (accuracy, precision, recall, f1-score)
accuracy_entropy_pruned = accuracy_score(y_test, y_pred_entropy_pruned)
precision_entropy_pruned = precision_score(y_test, y_pred_entropy_pruned, average='weighted')
recall_entropy_pruned = recall_score(y_test, y_pred_entropy_pruned, average='weighted')
f1_entropy_pruned = f1_score(y_test, y_pred_entropy_pruned, average='weighted')

# Mostrar las métricas
print("Métricas del Modelo Gini:")
print(f"Accuracy: {accuracy_gini:.4f}, Precision: {precision_gini:.4f}, Recall: {recall_gini:.4f}, F1-Score: {f1_gini:.4f}\n")
print("Métricas del Modelo Entropía:")
print(f"Accuracy: {accuracy_entropy:.4f}, Precision: {precision_entropy:.4f}, Recall: {recall_entropy:.4f}, F1-Score: {f1_entropy:.4f}\n")
print("Métricas del Modelo Entropía Podado:")
print(f"Accuracy: {accuracy_entropy_pruned:.4f}, Precision: {precision_entropy_pruned:.4f}, Recall: {recall_entropy_pruned:.4f}, F1-Score: {f1_entropy_pruned:.4f}\n")



Tamaño original: (5200, 11)
Tamaño sin outliers: (5055, 11)
Métricas del Modelo Gini:
Accuracy: 0.7883, Precision: 0.8018, Recall: 0.7883, F1-Score: 0.7908

Métricas del Modelo Entropía:
Accuracy: 0.7943, Precision: 0.8060, Recall: 0.7943, F1-Score: 0.7945

Métricas del Modelo Entropía Podado:
Accuracy: 0.7943, Precision: 0.8060, Recall: 0.7943, F1-Score: 0.7945



In [ ]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

## Conclusión:

La eliminación de outliers mejoró las métricas del modelo, indicando que los datos atípicos afectaban su desempeño. Al limpiar el conjunto de datos, los árboles de decisión lograron una clasificación más precisa y estable. Sin embargo, las métricas siguen siendo las mismas que cuando las varibles no estaban normalizadas.

#### 8. Árbol de Decisiones - CC:SI - ED:SI - Outliers:SI - Balanceo: SI

In [41]:
# datos a utilizar
data_model_7 = data.copy()

# ==========================
# Eliminar outliers (IQR)
# ==========================

# seleccionar solo las columnas numéricas
num_cols = data_model_7.select_dtypes(include=['float64', 'int64']).columns

# calcular Q1, Q3 y el rango intercuartílico (IQR)
Q1 = data_model_7[num_cols].quantile(0.25)
Q3 = data_model_7[num_cols].quantile(0.75)
IQR = Q3 - Q1

# crear una máscara booleana que identifique las filas SIN outliers
mask = ~((data_model_7[num_cols] < (Q1 - 1.5 * IQR)) |
         (data_model_7[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)

# filtrar los datos limpios
data_clean = data_model_7[mask].reset_index(drop=True)

print("Tamaño original:", data_model_7.shape)
print("Tamaño sin outliers:", data_clean.shape)

# ==========================
# Separar X e y
# ==========================
X = data_clean.drop("Workout_Type", axis=1)
y = data_clean["Workout_Type"]

# Dividir los datos en conjuntos de entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5200, stratify=y)

# 🔹 Escalado de las características (excepto 'Gender' y 'Workout_Type')
scaler = StandardScaler()

# Seleccionar columnas numéricas que no sean categóricas
numeric_cols = [col for col in X_train.columns if col not in ['Gender', 'Workout_Type']]

# Hacer copias para no perder las columnas categóricas
X_train_scaled = X_train.copy()
X_test_scaled = X_test.copy()

# Escalar solo las columnas numéricas
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

# Se crea un árbol con el criterio de impureza de Gini y una profundidad máxima de 5
modelo_gini = DecisionTreeClassifier(criterion='gini', max_depth=5, random_state=5200, class_weight='balanced')
# Se entrena el modelo con los datos de entrenamiento
modelo_gini.fit(X_train_scaled, y_train)

# Se crea un árbol con el criterio de Entropía
modelo_entropy = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=5200, class_weight='balanced')
# Se entrena el modelo
modelo_entropy.fit(X_train_scaled, y_train)

# Se crea un árbol con Entropía y una condición de poda adicional
modelo_entropy_pruned = DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_split=5, random_state=5200, class_weight='balanced')
# Se entrena el modelo
modelo_entropy_pruned.fit(X_train_scaled, y_train)

# Predecir con el modelo Gini
y_pred_gini = modelo_gini.predict(X_test_scaled)

# Predecir con el modelo de Entropía
y_pred_entropy = modelo_entropy.predict(X_test_scaled)

# Predecir con el modelo de Entropía podado
y_pred_entropy_pruned = modelo_entropy_pruned.predict(X_test_scaled)

# Métricas para el modelo Gini (accuracy, precision, recall, f1-score)
accuracy_gini = accuracy_score(y_test, y_pred_gini)
precision_gini = precision_score(y_test, y_pred_gini, average='weighted')
recall_gini = recall_score(y_test, y_pred_gini, average='weighted')
f1_gini = f1_score(y_test, y_pred_gini, average='weighted')

# Métricas para el modelo de Entropía (accuracy, precision, recall, f1-score)
accuracy_entropy = accuracy_score(y_test, y_pred_entropy)
precision_entropy = precision_score(y_test, y_pred_entropy, average='weighted')
recall_entropy = recall_score(y_test, y_pred_entropy, average='weighted')
f1_entropy = f1_score(y_test, y_pred_entropy, average='weighted')

# Métricas para el modelo de Entropía podado (accuracy, precision, recall, f1-score)
accuracy_entropy_pruned = accuracy_score(y_test, y_pred_entropy_pruned)
precision_entropy_pruned = precision_score(y_test, y_pred_entropy_pruned, average='weighted')
recall_entropy_pruned = recall_score(y_test, y_pred_entropy_pruned, average='weighted')
f1_entropy_pruned = f1_score(y_test, y_pred_entropy_pruned, average='weighted')

# Mostrar las métricas
print("Métricas del Modelo Gini:")
print(f"Accuracy: {accuracy_gini:.4f}, Precision: {precision_gini:.4f}, Recall: {recall_gini:.4f}, F1-Score: {f1_gini:.4f}\n")
print("Métricas del Modelo Entropía:")
print(f"Accuracy: {accuracy_entropy:.4f}, Precision: {precision_entropy:.4f}, Recall: {recall_entropy:.4f}, F1-Score: {f1_entropy:.4f}\n")
print("Métricas del Modelo Entropía Podado:")
print(f"Accuracy: {accuracy_entropy_pruned:.4f}, Precision: {precision_entropy_pruned:.4f}, Recall: {recall_entropy_pruned:.4f}, F1-Score: {f1_entropy_pruned:.4f}\n")



Tamaño original: (5200, 11)
Tamaño sin outliers: (5055, 11)
Métricas del Modelo Gini:
Accuracy: 0.7883, Precision: 0.8018, Recall: 0.7883, F1-Score: 0.7908

Métricas del Modelo Entropía:
Accuracy: 0.8002, Precision: 0.8114, Recall: 0.8002, F1-Score: 0.8018

Métricas del Modelo Entropía Podado:
Accuracy: 0.8002, Precision: 0.8114, Recall: 0.8002, F1-Score: 0.8018



In [ ]:
# TODO guardar métricas en el diccionario
# TODO hacer la importancia de variables y gráficar el arbol gini

## Conclusión:

Con el balanceo de clases y la eliminación de outliers, el modelo mantuvo métricas similares a las obtenidas sin escalar las variables. Aunque el modelo con entropía logró un buen desempeño (accuracy = 0.8002, precisión = 0.8114, recall = 0.8002 y F1-score = 0.8018), el escalado no tuvo impacto, ya que los árboles de decisión no dependen de la escala de las variables.